In [ ]:
import osimport pdfplumberimport refrom pdfminer.high_level import extract_textfrom tqdm import tqdm

In [ ]:
def extract_pdf_data(pdf_path):    """    Извлекает из PDF-файла:    1. Основной вид деятельности    2. ИНН    3. Состояние записи (действующая/прекращённая)    Возвращает словарь с результатами    """    result = {        "activity": None,        "inn": None,        "state": "Действующая"  # По умолчанию считаем запись действующей    }    termination_keywords = [        "прекращение", "прекращении", "прекращена", "прекратить",        "исключение", "исключении", "исключена",        "аннулирование", "аннулировании", "аннулирована",        "ликвидация", "ликвидации", "ликвидирована"    ]    try:        # Вариант 1: Извлечение через таблицы        with pdfplumber.open(pdf_path) as pdf:            found_main_activity_section = False            found_egrip_records = False            for page in pdf.pages:                # Извлекаем все таблицы со страницы                tables = page.extract_tables()                for table in tables:                    for row in table:                        # Удаляем лишние пробелы и пустые значения                        cleaned_row = [str(cell).strip() for cell in row if cell]                        if not cleaned_row:                            continue                        # Поиск начала раздела с основным видом деятельности                        if "основном виде деятельности" in " ".join(cleaned_row).lower():                            found_main_activity_section = True                            continue                        # В разделе основного вида деятельности ищем строку с кодом                        if found_main_activity_section:                            if any("код и наименование" in cell.lower() for cell in cleaned_row):                                # Значение обычно в последней колонке                                for cell in reversed(cleaned_row):                                    if cell and not cell.lower().startswith(("код", "сведения")):                                        result["activity"] = cell                                        found_main_activity_section = False                                        break                        # Ищем ИНН (строка 11)                        if cleaned_row[0] == '11' and not result["inn"]:                            # Ищем значение, похожее на ИНН (10-12 цифр)                            for cell in cleaned_row:                                if re.match(r'^\d{10,12}$', cell):                                    result["inn"] = cell                                    break                        # Альтернативный поиск ИНН по названию                        if not result["inn"] and any("инн" in cell.lower() for cell in cleaned_row):                            for cell in cleaned_row:                                if re.match(r'^\d{10,12}$', cell):                                    result["inn"] = cell                                    break                        # Поиск раздела с записями ЕГРИП                        if "сведения о записях" in " ".join(cleaned_row).lower() and "внесенных в егрип" in " ".join(                                cleaned_row).lower():                            found_egrip_records = True                            continue                        # В разделе записей ЕГРИП ищем признаки прекращения деятельности                        if found_egrip_records:                            # Проверяем, содержит ли причина записи ключевые слова о прекращении                            if any(kw in " ".join(cleaned_row).lower() for kw in termination_keywords):                                result["state"] = "Прекращённая"                                found_egrip_records = False        # Если через таблицы не нашли нужные данные, используем текстовый поиск        full_text = extract_text(pdf_path)        # Поиск основного вида деятельности через раздел        if not result["activity"]:            # Шаблон 1: Ищем заголовок раздела и последующее описание            activity_pattern = re.compile(                r'Сведения об основном виде деятельности.*?'                r'Код и наименование вида деятельности\s*[|:]\s*(.+?)\s*'                r'(?=\d+\s*[|]|Сведения|$)',                re.IGNORECASE | re.DOTALL            )            match = activity_pattern.search(full_text)            if match:                result["activity"] = match.group(1).strip()            # Шаблон 2: Ищем непосредственно значение после заголовка            if not result["activity"]:                activity_fallback = re.search(                    r'основном виде деятельности.*?'                    r'(01\.\d{2,3}\s*[-\s]?\s*.+?)\s*'                    r'(?=\d+\.\d{2,3}|Сведения|$)',                    full_text, re.IGNORECASE | re.DOTALL                )                if activity_fallback:                    result["activity"] = activity_fallback.group(1).strip()        # Поиск ИНН (если не нашли в таблицах)        if not result["inn"]:            inn_pattern = re.compile(                r'Идентификационный номер налогоплательщика \(?ИНН\)?\s*[|:]\s*(\d{10,12})',                re.IGNORECASE            )            match = inn_pattern.search(full_text)            if match:                result["inn"] = match.group(1).strip()        # Поиск признаков прекращения деятельности (если не нашли в таблицах)        if result["state"] == "Действующая":            # Шаблон 1: Поиск в разделе записей ЕГРИП            termination_pattern = re.compile(                r'Сведения о записях, внесенных в ЕГРИП.*?Причина внесения записи в ЕГРИП\s*[|:]\s*(.*?прекращ.*?)(?:\n|$)',                re.IGNORECASE | re.DOTALL            )            match = termination_pattern.search(full_text)            if match:                result["state"] = "Прекращённая"            else:                # Шаблон 2: Общий поиск по ключевым словам                if any(kw in full_text.lower() for kw in termination_keywords):                    result["state"] = "Прекращённая"    except Exception as e:        print(f"Ошибка при обработке {pdf_path}: {str(e)}")    return result

In [ ]:
def process_folder(folder_path):    """Обрабатывает все PDF-файлы в папке и выводит результаты"""    # Создаем список для результатов    results = []    # Получаем список файлов    files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]    # Обрабатываем файлы с прогресс-баром    for filename in tqdm(files, desc="Обработка PDF-файлов"):        filepath = os.path.join(folder_path, filename)        data = extract_pdf_data(filepath)        results.append({            "filename": filename,            "activity": data["activity"],            "inn": data["inn"],            "state": data["state"]        })    # Выводим результаты в удобном формате    print("\n" + "=" * 80)    print("Результаты обработки файлов:")    print("=" * 80)    for res in results:        print(f"Файл: {res['filename']}")        print(f"Основной вид деятельности: {res['activity'] or 'НЕ НАЙДЕН'}")        print(f"ИНН: {res['inn'] or 'НЕ НАЙДЕН'}")        print(f"Состояние записи: {res['state']}")        print("-" * 80)    # Сохраняем результаты в CSV-файл    csv_path = os.path.join(folder_path, "extracted_data.csv")    try:        with open(csv_path, "w", encoding="utf-8") as f:            f.write("Имя файла;Основной вид деятельности;ИНН;Состояние записи\n")            for res in results:                f.write(f"{res['filename']};{res['activity'] or ''};{res['inn'] or ''};{res['state']}\n")        print(f"\nРезультаты сохранены в файл: {csv_path}")    except Exception as e:        print(f"Ошибка при сохранении CSV: {str(e)}")

In [ ]:
# Укажите путь к вашей папке с PDF-файламиfolder_path = "отчеты"process_folder(folder_path)